In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nbimporter

import read_conditions as rc
import analyze_control as ac
import central_system as cs

import report_study_case

Importing Jupyter notebook from report_study_case.ipynb


# 住宅の仕様

本モジュールでは、住宅の仕様の計算について示します。

## 0. 検討ケースの取得

In [2]:
case = report_study_case.get_case()
case['house']

{'a_mr': 29.81,
 'a_or': 51.34,
 'a_a': 120.08,
 'r_env': 2.9555555555555557,
 'insulation': 'h11',
 'solar_gain': 'middle'}

## 1. 面積

### 非居室

非居室の床面積 $A_{NR}$ は下式で表される。

$$ A_{NR} = A_A - A_{MR} - A_{OR}$$

$A_{NR}$：非居室の床面積, m<sup>2</sup>  
$A_A$：床面積の合計, m<sup>2</sup>  
$A_{MR}$：主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$：その他の居室の床面積, m<sup>2</sup>  

In [3]:
a_nr = cs.get_non_occupant_room_floor_area(case['house']['a_mr'], case['house']['a_or'], case['house']['a_a'], case['house']['r_env'])
a_nr

38.92999999999999

### 標準住戸及び当該住戸における暖冷房区画

標準住戸における暖冷房区画 $i$ の床面積 $A_{HCZ,R,i}$ 及び暖冷房区画 $i$ の床面積 $A_{HCZ,i}$ は第2章で計算される。 

In [4]:
a_hcz_r = cs.get_referenced_floor_area()
a_hcz_r

array([29.81, 16.56, 13.25, 10.76, 10.77,  3.31,  1.66,  3.31, 13.25,
        4.97, 10.77,  1.66])

In [5]:
a_hcz = cs.get_floor_area(case['house']['a_mr'], case['house']['a_or'], case['house']['a_a'], case['house']['r_env'])
a_hcz

array([29.81, 16.56, 13.25, 10.76, 10.77,  3.31,  1.66,  3.31, 13.25,
        4.97, 10.77,  1.66])

## 2. 間仕切り

### 面積

暖冷房空間 $i$ ( $i=1$ ) (主たる居室) から見た非暖冷房空間の間仕切りの面積
$$ A_{prt,i} = A_{HCZ,i} r_{env} \frac{ A_{NR} }{ A_{OR} + A_{NR} } $$

暖冷房空間 $i$ ( $i=2,3,4,5$ ) (その他の居室) から見た非暖冷房空間の間仕切りの面積
$$ A_{prt,i} = A_{HCZ,i} r_{env} \frac{ A_{NR} }{ A_{MR} + A_{NR} } $$

$A_{prt,i}$：暖冷房区画 $i$ から見た非暖冷房空間の間仕切りの面積, m<sup>2</sup>  
$A_{HCZ,i}$：暖冷房区画 $i$ の床面積, m<sup>2</sup>  
$r_{env}$：床面積の合計に対する外皮の部位の面積の合計の比  
$A_{MR}$：主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$：その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$：非居室の床面積, m<sup>2</sup>  

In [6]:
a_prt = cs.get_partition_area(a_hcz, case['house']['a_mr'], case['house']['a_or'], a_nr, case['house']['r_env'])
a_prt

array([37.99636619, 27.7187943 , 22.17838312, 18.01052093, 18.02725933])

### 熱損失係数

間仕切りの熱貫流率 $U_{prt}$ は 1/0.46 とする

In [7]:
u_prt = cs.get_heat_loss_coefficient_of_partition()
u_prt

2.1739130434782608

## 3. 外皮性能

本来これは入力値である。（この notebook では、説明を簡単にするため、断熱レベル・日射熱取得レベルで指定するようにしている。）

In [8]:
q, mu_h, mu_c = cs.get_envelope_spec(case['region'], case['house']['insulation'], case['house']['solar_gain'])
q, mu_h, mu_c

(2.7, 0.101, 0.062)

## 4. 機械換気量

（この値は建築基準法等に則って、住宅の仕様のみから決定される値のため、本モジュールで計算する。）

$$V_{vent,i} = V_{vent,R,i} \frac{ A_{HCZ,i} }{ A_{HCZ,R,i} } $$

$V_{vent,i}$：暖冷房区画 $i$ の機械換気量, m<sup>3</sup>/h  
$V_{vent,R,i}$：標準住戸における暖冷房区画 $i$ の機械換気量, m<sup>3</sup>/h  
$A_{HCZ,i}$：暖冷房区画 $i$ の床面積, m<sup>2</sup>  
$A_{HCZ,R,i}$：標準住戸における暖冷房区画 $i$ の床面積, m<sup>2</sup>  

標準住戸における暖冷房区画 $i$ の機械換気量 $V_{vent,R,i}$ は下表で与えられる。

| 空間番号 | 標準住戸における暖冷房区画iの機械換気量 |
| --- | --- |
| 1 | 60.0 |
| 2 | 20.0 |
| 3 | 40.0 |
| 4 | 20.0 |
| 5 | 20.0 |

In [9]:
v_vent = cs.get_mechanical_ventilation(a_hcz_r, a_hcz)
v_vent

array([60., 20., 40., 20., 20.])

## 4. 外部モジュールへの引き渡し

本モジュールを外部のモジュールから呼び出して必要な値を取得する場合に利用する関数を定義します。

In [10]:
def get_house_spec(region, house_spec):
    
    # the ratio of the total envelope area to the total floor area
    r_env = house_spec['r_env']
    
    # total floor area, m2
    a_a = house_spec['a_a']
    
    # main occupant room floor area, m2
    a_mr = house_spec['a_mr']
    
    # other occupant room floor area, m2
    a_or = house_spec['a_or']
    
    # non occupant room floor area, m2
    a_nr = cs.get_non_occupant_room_floor_area(a_mr, a_or, a_a, r_env)
    
    # heating and cooling zone floor area of referenced house, m2
    a_hcz_r = cs.get_referenced_floor_area()
    
    # heating and cooling zone floor area, m2 
    a_hcz = cs.get_floor_area(a_mr, a_or, a_a, r_env)

    # partition wall area, m2
    a_prt = cs.get_partition_area(a_hcz, a_mr, a_or, a_nr, r_env)
    
    # partition wall u value, W/m2K
    u_prt = cs.get_heat_loss_coefficient_of_partition()
    
    # q value, W/m2K, mu value
    q, mu_h, mu_c = cs.get_envelope_spec(region, house_spec['insulation'], house_spec['solar_gain'])
    
    # required mechanical ventilation volume, m3/h 
    v_vent = cs.get_mechanical_ventilation(a_hcz_r, a_hcz)
    
    return r_env, a_a, a_mr, a_or, a_nr, a_hcz_r, a_hcz, a_prt, u_prt, q, mu_h, mu_c, v_vent
